# 1 часть

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator

spark = SparkSession.builder.master("local[*]").getOrCreate()
filename_data = 'datasetautoru.csv'
csv = spark.read.csv(filename_data, inferSchema=True, header=True)
csv = csv.withColumn('mileage', csv.mileage.cast(IntegerType()))
csv.show(10)

+---+----------+---------+----------+----+-------+-----------------+---------+------------+--------------+--------+------+------------+---------+
|_c0|     brand|    model|  offer_id|year|mileage|        body_type|fuel_type|engine_power|  transmission|   drive| wheel|owners_count|    price|
+---+----------+---------+----------+----+-------+-----------------+---------+------------+--------------+--------+------+------------+---------+
|  0|    Toyota|     Vitz|1102756899|2001|   6236|    хэтчбек 5 дв.|   Бензин|        70.0|автоматическая|передний|Правый|           2| 230000.0|
|  1|LADA (ВАЗ)|     2107|1102754237|1995|  96000|            седан|   Бензин|        75.0|  механическая|  задний| Левый|           2|  35000.0|
|  2|    Daewoo|   Nubira|1102763224|1998|  50000|            седан|   Бензин|       106.0|  механическая|передний| Левый|           3|  45000.0|
|  3| Chevrolet|     Aveo|1102757199|2014| 119120|            седан|   Бензин|       115.0|  механическая|передний| Левый|  

In [2]:
splits = csv.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("price", "truePrice")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 192250  Testing Rows: 82504


In [3]:
strIdx = StringIndexer(inputCols = ['brand', 'model', 'fuel_type','transmission', 'wheel', 'body_type', 'drive'], 
                       outputCols = ['brand_index', 'model_index', 'fuel_type_index', 'transmission_index', 'wheel_index', 'body_type_index', 'drive_index'], 
                       handleInvalid = "keep")
catVect = VectorAssembler(inputCols = ['brand_index', 'model_index', 'fuel_type_index','transmission_index', 'wheel_index', 'body_type_index', 'drive_index'],
                          outputCol="features_cat")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), 
                       outputCol = "features_index", 
                       handleInvalid = "keep")
numVect = VectorAssembler(inputCols = ["mileage",'engine_power', 'year', 'owners_count'], 
                          outputCol="features_num", 
                          handleInvalid = "keep")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), 
                      outputCol="features_norm")
featVect = VectorAssembler(inputCols=["features_index", "features_norm"], 
                           outputCol="features", 
                           handleInvalid = "keep")
rfr = RandomForestRegressor(featuresCol = 'features', 
                      labelCol='price',
                      numTrees = 10,
                      maxDepth=2,
                      maxBins = 181834)
pipeline = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect, rfr])

In [4]:
pipelineModel = pipeline.fit(train)

In [5]:
pred_df = pipelineModel.transform(test)
pred_df.select("features", "prediction", "truePrice").show()

+--------------------+------------------+---------+
|            features|        prediction|truePrice|
+--------------------+------------------+---------+
|[2.0,150.0,0.0,1....|307290.46202095674| 230000.0|
|(11,[1,7,8,9,10],...|194551.19352775178| 120000.0|
|[4.0,88.0,0.0,1.0...| 755219.8640500369|1199000.0|
|[3.0,380.0,1.0,0....| 543984.0969596596| 750000.0|
|[5.0,47.0,0.0,3.0...| 564796.0376066513| 590000.0|
|(11,[1,5,7,8,9],[...| 548764.9315557991| 600000.0|
|(11,[1,5,7,8,9,10...|238497.69436954666| 189990.0|
|[4.0,29.0,0.0,0.0...|  653059.894275976| 875000.0|
|(11,[0,1,7,8,9,10...|302182.00840575853| 250000.0|
|[9.0,8.0,0.0,0.0,...|467877.70219168655| 470000.0|
|(11,[5,7,8,9],[3....| 414720.5508152448| 539990.0|
|(11,[0,1,7,8,9,10...|245600.89639284168| 190000.0|
|(11,[1,5,7,8,9,10...| 206418.4108570427|  75000.0|
|[12.0,14.0,0.0,0....|350618.57677717775| 240000.0|
|[3.0,30.0,0.0,2.0...| 755219.8640500369|1280000.0|
|[1.0,56.0,0.0,1.0...| 755219.8640500369| 990000.0|
|[45.0,303.0

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator

regressionEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="truePrice", metricName="rmse")

In [7]:
# RMSE
rmse = regressionEvaluator.evaluate(pred_df)
print(f"The RMSE for the random forest regression model is {rmse:0.2f}")
# MSE
mse = regressionEvaluator.setMetricName("mse").evaluate(pred_df)
print(f"The MSE for the random forest regression model is {mse:0.2f}")
# R2
r2 = regressionEvaluator.setMetricName("r2").evaluate(pred_df)
print(f"The R2 for the random forest regression model is {r2:0.2f}")
# MAE
mae = regressionEvaluator.setMetricName("mae").evaluate(pred_df)
print(f"The MAE for the random forest regression model is {mae:0.2f}")

The RMSE for the random forest regression model is 194608.39
The MSE for the random forest regression model is 37872427164.22
The R2 for the random forest regression model is 0.63
The MAE for the random forest regression model is 143126.63


In [8]:
param_grid = ParamGridBuilder().\
    addGrid(rfr.numTrees, [10, 15, 20]).\
    addGrid(rfr.maxDepth, [1, 2, 4]).\
    addGrid(rfr.maxBins , [181834, 362432, 724864]).\
    build()

In [9]:
cv = CrossValidator(estimator=pipeline, \
                    estimatorParamMaps=param_grid, \
                    evaluator=RegressionEvaluator(
                                predictionCol="prediction", \
                                labelCol="price", \
                                metricName="rmse"), \
                    numFolds=2)

In [10]:
cv_model = cv.fit(train)

In [11]:
newPrediction = cv_model.transform(test)

In [12]:
# RMSE
rmse = regressionEvaluator.evaluate(newPrediction)
print(f"The RMSE for the random forest regression model is {rmse:0.2f}")
# MSE
mse = regressionEvaluator.setMetricName("mse").evaluate(newPrediction)
print(f"The MSE for the random forest regression model is {mse:0.2f}")
# R2
r2 = regressionEvaluator.setMetricName("r2").evaluate(newPrediction)
print(f"The R2 for the random forest regression model is {r2:0.2f}")
# MAE
mae = regressionEvaluator.setMetricName("mae").evaluate(newPrediction)
print(f"The MAE for the random forest regression model is {mae:0.2f}")

The RMSE for the random forest regression model is 106422.51
The MSE for the random forest regression model is 23174072558.53
The R2 for the random forest regression model is 0.77
The MAE for the random forest regression model is 106422.51


# 2 часть

In [13]:
csv = csv.drop(csv._c0).withColumn('label', when(col('mileage') >= 150942.9, 1).otherwise(0))
csv = csv.drop(csv.mileage)
csv.show()

+----------+---------+----------+----+-----------------+---------+------------+----------------+--------+------+------------+-----------------+-----+
|     brand|    model|  offer_id|year|        body_type|fuel_type|engine_power|    transmission|   drive| wheel|owners_count|            price|label|
+----------+---------+----------+----+-----------------+---------+------------+----------------+--------+------+------------+-----------------+-----+
|    Toyota|     Vitz|1102756899|2001|    хэтчбек 5 дв.|   Бензин|        70.0|  автоматическая|передний|Правый|           2|         230000.0|    0|
|LADA (ВАЗ)|     2107|1102754237|1995|            седан|   Бензин|        75.0|    механическая|  задний| Левый|           2|          35000.0|    0|
|    Daewoo|   Nubira|1102763224|1998|            седан|   Бензин|       106.0|    механическая|передний| Левый|           3|          45000.0|    0|
| Chevrolet|     Aveo|1102757199|2014|            седан|   Бензин|       115.0|    механическая|пере

In [14]:
splits = csv.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
print("Training Rows:", train.count(), " Testing Rows:", test.count())

Training Rows: 192580  Testing Rows: 82174


In [15]:
strIdx = StringIndexer(inputCols = ['brand', 'model', 'fuel_type','transmission', 'wheel', 'body_type', 'drive'], 
                       outputCols = ['brand_index', 'model_index', 'fuel_type_index', 'transmission_index', 'wheel_index', 'body_type_index', 'drive_index'], 
                       handleInvalid = "keep")
catVect = VectorAssembler(inputCols = ['brand_index', 'model_index', 'fuel_type_index', 'transmission_index', 'wheel_index', 'body_type_index', 'drive_index'], 
                          outputCol="features_cat")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), 
                       outputCol = "features_index", 
                       handleInvalid = "keep")
numVect = VectorAssembler(inputCols = ['engine_power', 'year', 'owners_count'], 
                          outputCol="features_num", 
                          handleInvalid = "keep")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), 
                      outputCol="features_norm")
featVect = VectorAssembler(inputCols=["features_index", "features_norm"], 
                           outputCol="features", 
                           handleInvalid = "keep")
lr = LogisticRegression(labelCol="label", 
                        featuresCol="features", 
                        maxIter=10,
                        regParam=0.3)
pipeline = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect, lr])

In [16]:
pipelineModel = pipeline.fit(train)

In [17]:
pred_df = pipelineModel.transform(test)
pred_df.select("features", "prediction", "trueLabel").show()

+--------------------+----------+---------+
|            features|prediction|trueLabel|
+--------------------+----------+---------+
|[56.0,720.0,1.0,0...|       1.0|        1|
|[56.0,1266.0,0.0,...|       1.0|        0|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8],[17...|       0.0|        1|
|[17.0,128.0,0.0,0...|       1.0|        0|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|[17.0,128.0,1.0,0...|       0.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8],[17...|       0.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|[17.0,123.0,0.0,1...|       1.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8,9],[...|       1.0|        1|
|(10,[0,1,7,8,9],[...|       1.0

In [18]:
tp = float(pred_df.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(pred_df.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(pred_df.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(pred_df.filter("prediction == 0.0 AND truelabel == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|           19469.0|
|       FP|            7323.0|
|       TN|           38105.0|
|       FN|           17277.0|
|Precision|0.7266721409375934|
|   Recall|0.5298263756599357|
|       F1|0.6128301174100539|
+---------+------------------+



In [19]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(pred_df)
print ("AUR = ", aur)

AUR =  0.775801742331711


In [20]:
paramGrid = ParamGridBuilder().\
    addGrid(lr.maxIter, [30, 40, 60]).\
    addGrid(lr.regParam, [0.6, 0.8, 0.9]).build()
cv = CrossValidator(estimator=pipeline, evaluator=BinaryClassificationEvaluator(metricName='areaUnderPR'), estimatorParamMaps=paramGrid, 
                    numFolds=2)

In [21]:
cv_model = cv.fit(train)

In [22]:
newPrediction = cv_model.transform(test)

In [23]:
# Recalculate confusion matrix
tp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 1").count())
fp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 0").count())
tn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 0").count())
fn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 1").count())
pr2 = tp2 / (tp2 + fp2)
re2 = tp2 / (tp2 + fn2)
metrics2 = spark.createDataFrame([
 ("TP", tp2),
 ("FP", fp2),
 ("TN", tn2),
 ("FN", fn2),
 ("Precision", pr2),
 ("Recall", re2),
 ("F1", 2*pr2*re2/(re2+pr2))],["metric", "value"])
metrics2.show()

+---------+-------------------+
|   metric|              value|
+---------+-------------------+
|       TP|            14086.0|
|       FP|             4737.0|
|       TN|            40691.0|
|       FN|            22660.0|
|Precision| 0.7483397970567922|
|   Recall|0.38333424046154685|
|       F1| 0.5069733124583851|
+---------+-------------------+



In [24]:
# Recalculate the Area Under ROC
evaluator2 = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
aur2 = evaluator2.evaluate(newPrediction)
print( "AUR2 = ", aur2)

AUR2 =  0.6395296719609839
